In [7]:
json_objects = {
  "nodes": {
    "__START__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "Entry point of the graph.",
      "function_name": ""
    },
    "planner": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field",
      "function_name": "plan_step"
    },
    "agent": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "PlanExecute",
      "description": "Uses llm with tool binding for the stock related queries",
      "function_name": "execute_step"
    },
    "replan": {
      "schema_info": "PlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)",
      "input_schema": "PlanExecute",
      "output_schema": "Union[Response, Plan]",
      "description": "Evaluates progress and uses an LLM to either revise the plan or generate a final response.",
      "function_name": "replan_step"
    },
    "__END__": {
      "schema_info": "",
      "input_schema": "",
      "output_schema": "",
      "description": "End point of the graph.",
      "function_name": ""
    }
  },
  "edges": {
    "edge_1": {
      "source": "__START__",
      "target": "planner",
      "routing_conditions": "Start the planning process.",
      "conditional": False
    },
    "edge_2": {
      "source": "planner",
      "target": "agent",
      "routing_conditions": "After planning, execute the first step.",
      "conditional": False
    },
    "edge_3": {
      "source": "agent",
      "target": "replan",
      "routing_conditions": "After executing a step, check if replanning is needed.",
      "conditional": False
    },
    "edge_4": {
      "source": "replan",
      "target": "agent",
      "routing_conditions": "If no response is generated, continue to agent for further execution.",
      "conditional": True
    },
    "edge_5": {
      "source": "replan",
      "target": "__END__",
      "routing_conditions": "If a response is generated, end the process.",
      "conditional": True
    }
  }
}

In [8]:
import import_ipynb
from phase1_edge_handling import edge_builder_agent
from phase1_node_to_code import node_to_code_app

In [9]:
# node name to corresponding code stubs
node_to_code = {}

# edge_to_code 
# key will be src#dest
# contains code snippets generated.
edge_to_code = {}

skipList = ["__START__", "__END__"]

In [10]:
import uuid
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.messages import HumanMessage
edge_info_prompt = ChatPromptTemplate.from_template("""
<GraphNodeImplementation>
{node_code}
</GraphNodeImplementation>
<EdgeInformation>
{edge_json}
</Edgeinformation>""")

def dfs_traverse(json_objects, current_node, visited=None):
    if visited is None:
        visited = set()  # Initialize the visited set

    # If we encounter a node we've already visited, we skip to avoid infinite loops
    if current_node in visited:
        return

    visited.add(current_node)  # Mark the current node as visited
    print(f"Visiting node: {current_node}")

    # Get edges originating from the current node
    edges = json_objects["edges"]
    nodes = json_objects["nodes"]
    outgoing_edges = list(filter(lambda edge: edge["source"] == current_node, edges.values()))
    
    if current_node not in skipList:
        uuid_str = uuid.uuid4()
        config = {"configurable": {"thread_id": str(uuid_str)}}

        for output in node_to_code_app.stream(nodes[current_node], config, stream_mode="updates"):
            print(output)
        code= node_to_code_app.get_state(config).values["final_code"]
        result = edge_builder_agent.invoke({"messages": [HumanMessage(content = edge_info_prompt.format(node_code=code,edge_json=outgoing_edges))]},config)
        node_to_code[current_node] = code
        edge_to_code[current_node] = result["messages"][-1].content
    for edge_key, edge_value in edges.items():
        if edge_value["source"] == current_node:  # Check if the edge originates from the current node
            next_node = edge_value["target"]
            print(f"Following edge from {current_node} to {next_node}")
            dfs_traverse(json_objects, next_node, visited)  # Recursively call DFS on the next node

In [11]:
dfs_traverse(json_objects, "__START__")

Visiting node: __START__
Following edge from __START__ to planner
Visiting node: planner
{'identify_node': {'node_type': 'planner', 'messages': [HumanMessage(content='\nYou are provided with the following information about the node:\n<SchemaInfo>\nPlanExecute: TypedDict with fields input (str), plan (List[str]), past_steps (Annotated[List[Tuple], operator.add]), response (str)\n</SchemaInfo>\n<InputSchema>\nPlanExecute\n</InputSchema>\n<OutputSchema>\nPlanExecute\n</OutputSchema>\n<Description>\nPlan step generates a plan based on the input using llm structured output functionality, stores it to the plan field\n</Description>\n<FunctionName>\nplan_step\n</FunctionName>\n\nBelow is the skeleton of the function that you need to implement:\ndef plan_step(state:PlanExecute) -> PlanExecute:\n    """Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field"""\n    # Implement the function to meet the description.\n\nthe state is of t

In [14]:
for edge_key, edge_val in edge_to_code.items():
    print(f"################{edge_key}##########")
    print(edge_val)
    print("####################################################")

################planner##########
The provided edge information indicates that this is a non-conditional edge that directly routes from the 'planner' node to the 'agent' node after the planning step is completed. Here’s the Python code snippet that implements the edge:

```python
# Assuming we have a 'graph' object already defined

# Adding a non-conditional edge from the 'planner' node to the 'agent' node
graph.add_edge("planner", "agent")

# This edge defines a direct path from 'planner' to 'agent', which implies 
# that once the planning step is complete, the execution will immediately 
# move to the 'agent' node for further action.
```

This snippet demonstrates the implementation of a non-conditional edge as specified, ensuring that after executing the 'planner' node, the flow continues to the 'agent' node without any conditions.
####################################################
################agent##########
```python
# Implementing a non-conditional edge from "agent" to "rep

In [15]:
for node_key, node_val in node_to_code.items():
    print(f"################{node_key}##########")
    print(node_val)
    print("####################################################")

################planner##########
```python
from typing import List, Tuple, TypedDict
import operator

# Define the TypedDict as per the schema information
class PlanExecute(TypedDict):
    input: str
    plan: List[str]
    past_steps: List[Tuple]
    response: str

def plan_step(state: PlanExecute) -> PlanExecute:
    """Plan step generates a plan based on the input using llm structured output functionality, stores it to the plan field."""
    
    # Extracting input from the state
    input_data = state['input']
    
    # Generating the prompt for the LLM
    prompt = (
        "Hello, ChatGPT! I hope you are doing well. I am reaching out to you for assistance with generating a structured plan based on specific input I will provide. \n\n"
        "Please find the details below:\n\n"
        "**function_name:** [generate_plan]  \n"
        f"**input:** {input_data}  \n"
        "**rule:** [I want you to act as a planning assistant. Based on the input, please create a detailed, step-